In [4]:
import pandas as pd
import numpy as np

In [10]:
# Bibliotecas a importar
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, classification_report

In [12]:
# https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data
# Abrir o Data Set Breast Cancer
df = pd.read_csv('data.csv')
df

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN


In [14]:
# Separar o dataframe em X e y
X = df.iloc[:, 2:-1] #selecionar todas as linhas e começar a partir da 3 coluna menos a ultima
y = df.iloc[:, 1]
y = y.replace ({'B':0, 'M':1})

C:\Users\Guuto\AppData\Local\Temp\ipykernel_23492\2838307123.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace ({'B':0, 'M':1})


In [16]:
# Divisão treino e teste
# Separar 70% treino e 30% teste, declarando o seed random
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [18]:
# Dimensões dos datasets X_train, X_test, y_train, y_test após a divisão em treino e teste (formado linhas e colunas)
print("Dimensões de X_train:", X_train.shape)
print("Dimensões de X_test:", X_test.shape)
print("Dimensões de y_train:", y_train.shape)
print("Dimensões de y_test:", y_test.shape)

Dimensões de X_train: (398, 30)
Dimensões de X_test: (171, 30)
Dimensões de y_train: (398,)
Dimensões de y_test: (171,)


In [20]:
# Redução Dimensionalidade com PCA
# Primeiro devemos aplicar o Standard Scaler, usando fit_transform na base de treino, mas apenas transform na base de teste
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [22]:
# Aplicar a transformação com PCA nos dados escalonados:
pca = PCA(n_components=10, random_state=10)

pca.fit(X_train_scaled)

PCA(n_components=10, random_state=10)

In [24]:
# Analisar a explicabilidade das variâncias nos dados:
pca.explained_variance_ratio_

array([0.4383292 , 0.18413908, 0.10158532, 0.06803093, 0.05624249,
       0.03953245, 0.02437355, 0.01630403, 0.01462998, 0.01175767])

In [26]:
# Soma acumulada da variância explicada
explained_variance_cumsum = np.cumsum(pca.explained_variance_ratio_)

# Econtrando o número mínimo de componentes que explicam pelo menos 70% da variância
n_components_70 = np.argmax(explained_variance_cumsum >= 0.70) + 1 # +1 porque índices começam em 0

print(f"Número de componentes necessários da variância de 70%: {n_components_70}")

Número de componentes necessários da variância de 70%: 3


In [133]:
# Criar os datasets X_train_pca e X_test_pca, transformados pelo PCA.
# Utilizando o número de componentes que representam 70% de variância dos dados

In [28]:
pca = PCA(n_components=3, random_state=10)
pca.fit(X_train_scaled)
X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [30]:
# Dimensões dos datasets X_train_pca, X_test_pca (formado linhas e colunas)
print("Dimensões de X_train_pca:", X_train_pca.shape)
print("Dimensões de X_test_pca:", X_test_pca.shape)

Dimensões de X_train_pca: (398, 3)
Dimensões de X_test_pca: (171, 3)


In [32]:
# Busca de Hiperparâmetros com Validação Cruzada
# DecisionTreeClassifier

# Solução com GridSearchCV (Busca Exaustiva)
# Criar o classificador de árvore de decisão
estimator = DecisionTreeClassifier(random_state=10)

# Definição do grid de hiperparâmetros
param_grid = {
    "max_depth": [3, None],
    "min_samples_split": [2, 10]
}

# Configuração do GridSearchCV
grid = GridSearchCV(
    estimator=estimator,
    param_grid=param_grid,
    scoring="accuracy",  
    cv=5 
)

# Treinamento da busca
grid.fit(X_train_pca, y_train)

# Melhor combinação encontrada
print("Melhores hiperparâmetros:", grid.best_params_)
print("Melhor acurácia:", grid.best_score_)

Melhores hiperparâmetros: {'max_depth': 3, 'min_samples_split': 2}
Melhor acurácia: 0.9219620253164557


In [38]:
# Definição do modelo base
estimator = DecisionTreeClassifier(random_state=10)

# Configuração do RandomizedSearchCV
# Definição da distribuição de hiperparâmetros
param_distributions = {'max_depth': [3, None],
                                 'min_samples_split': [2, 10]}
number_interations = 2

grid = RandomizedSearchCV(
    estimator=estimator,
    param_distributions=param_distributions,
    scoring='accuracy',
    cv=5,
    n_iter=number_interations,
    random_state=10)

# Treinamento de busca
grid.fit(X_train_pca, y_train)

# Melhor combinação encontrada
print("Melhores hiperparâmetros:", grid.best_params_)
print("Melhores hiperparâmetros:", grid.best_score_)

Melhores hiperparâmetros: {'min_samples_split': 2, 'max_depth': 3}
Melhores hiperparâmetros: 0.9219620253164557


In [78]:
# Após a busca pelos hiperparâmetros, é possível analisar os resultados visualizando as saídas
# Mostra um DataFrame com todas as combinações testadas, incluindo acurácia média e desvio padrão para cada configuração.

pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_split,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003545,0.000406,0.001601,0.000490,2,3,"{'min_samples_split': 2, 'max_depth': 3}",0.9625,0.8750,0.925,0.962025,0.911392,0.927184,0.032980,1
1,0.003444,0.000475,0.001206,0.000401,10,3,"{'min_samples_split': 10, 'max_depth': 3}",0.9625,0.8750,0.925,0.962025,0.911392,0.927184,0.032980,1
2,0.004780,0.000862,0.001043,0.000069,2,None,"{'min_samples_split': 2, 'max_depth': None}",0.9375,0.8875,0.925,0.936709,0.924051,0.922152,0.018221,3
3,0.004279,0.000838,0.001083,0.000145,10,None,"{'min_samples_split': 10, 'max_depth': None}",0.9875,0.8625,0.925,0.936709,0.886076,0.919557,0.043166,4


In [80]:
# PASSO 11 - Resultados

print("Melhor índice:", grid.best_index_)
print("Melhor hiperparâmetros:", grid.best_params_)
print("Melhor acurácia:", grid.best_score_)
print("Melhor modelo:", grid.best_estimator_)

Melhor índice: 0
Melhor hiperparâmetros: {'min_samples_split': 2, 'max_depth': 3}
Melhor acurácia: 0.9271835443037976
Melhor modelo: DecisionTreeClassifier(max_depth=3, random_state=10)


In [44]:
# Método Ensemble
# Utilizar como modelo base nossa árvore de decisão com os melhores hiperparâmetros encontrados
estimator = DecisionTreeClassifier(
    random_state=10,
    max_depth=3,
    min_samples_split=2)

# Instancia em esemble Bagging
# Construir um modelo com 100 estimadores através da Instancia em Esemble Bagging  
esemble = BaggingClassifier(
    estimator=estimator,
    n_estimators=100,
    random_state=10)

# Treina o esemble Bagging
esemble.fit(X_train_pca, y_train)

# Previsão usando o Bagging
y_pred = esemble.predict(X_test_pca)

In [50]:
# Calcular a acurácia do modelo anterior
predictions = esemble.predict(X_test_pca)

accuracy = accuracy_score(y_test, predictions)

print("Acurácia:", accuracy)

Acurácia: 0.9590643274853801
